ESPACE DE RECHERCHE / HYPERPARAMETRE

In [2]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn=0.24.1
- pip:
  - numpy==1.19.5
  - pandas==0.25.3
  - azureml-defaults

from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

Overwriting conda_dependencies.yml


In [1]:
#-------------------------- ETAPE 1 : MODEL INIT

# connection au WS
from azureml.core import Workspace
ws = Workspace.get(subscription_id = "a0f4cddc-a66a-4dcc-9df7-ccbd7f81bf7b",
                   resource_group = "ressourcesecu",
                   name = "deuxiemeespace")

ws.get_details()

# Execution du script en tant qu'experience
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal,RandomParameterSampling

# Create a Python environment for the experiment
sklearn_env = Environment("sklearn-env")
# Ensure the required packages are installed
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','pandas'])
sklearn_env.python.conda_dependencies = packages

# Create a script config + target compute ressource
script_config = ScriptRunConfig(source_directory=".",
                                script='scripts/training_script_hyperparametres.py',
                                compute_target = 'jucalcul1',
                                environment=sklearn_env) 

# Submit the experiment
experiment = Experiment(workspace=ws, name='test_hyperparametres')
run  = experiment.submit(config=script_config)

from azureml.widgets import RunDetails
RunDetails(run).show()

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.33.0 (c:\users\jcharley\anaconda3\lib\site-packages), Requirement.parse('azureml-core~=1.32.0'), {'azureml-telemetry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-core 1.33.0 (c:\users\jcharley\anaconda3\lib\site-packages), Requirement.parse('azureml-core~=1.32.0'), {'azureml-telemetry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.33.0 (c:\users\jcharley\anaconda3\lib\site-packages), Requirement.parse('azureml-core~=1.32.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reuse

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
# save 
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = run.register_model(model_name='modele3', 
                           model_path='outputs/model.pkl',
                           model_framework=Model.Framework.SCIKITLEARN,
                           model_framework_version='0.24.1',
                           resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5))

# deploy
hosting_model = Model.deploy(ws, "modele3", [model])

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
#-------------------------- ETAPE 2 : TUNNING PARAMETERS
# Espace de recherche des hyperparamètres
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
    

param_sampling = RandomParameterSampling( {
    "--n_estimators": choice(15, 50, 100, 200, 300),
    "--criterion": choice("gini", "entropy")
    }
)


hyperdrive_config = HyperDriveConfig(run_config=script_config,
                                     hyperparameter_sampling=param_sampling, 
                                     primary_metric_name='accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=8,
                                     max_concurrent_runs=4)

# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

In [ ]:
for child_run in run.get_children():
    print(child_run.id, child_run.get_metrics())

In [ ]:
#-------------------------- ETAPE 3 : ENREGISTREMENT

#récuperer le plus performant
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

In [ ]:
#host / save the best one 
model = best_run.register_model(model_name='rf_tuning', model_path='outputs/model_rf.joblib')